# Boosting(부스팅)
- 단순하고 약한 학습기(Weak Learner)들 결합해, 정확하고 강력한 학습기(Strong Learner) 만드는 방식  
- 정확도가 낮은 하나의 모델을 만들어 학습시킨 뒤, 그 모델의 예측 오류는 두 번째 모델이 보완하는 방식
- 이 두 모델을 합치면 처음보다는 정확한 모델 만들어지며, 합쳐진 모델의 예측 오류는 다음 모델에서 보완하여 계속 더하는 과정 반복
- 약한 학습기들은 앞 학습기가 만든 오류 줄이는 방향으로 학습
- vs Voting
    - Voting 방식의 경우, 각 모델이 독립적(개별적)으로 학습
    - 각 모델의 결과를 취합해 최종 결과 산출
    - 병렬적으로 일할 수 있음
- gradient boosting(PDF 참고)
    - 처음 모델은 y 예측 -> 두번째 모델부터는 앞 모델이 만든 오류(실제 값 - 예측 값) 예측 -> 그것을 앞 모델에 업데이트하면 오류 줄일 수 있음
    - 오류를 update 할 때, 뺄지 더할지를 결정할 때, gradient descent 방법 사용. 미분해서 나오는 값의 음수를 취해서 적용 
    - 학습률(Learning Rate)을 작게 하면 update를 조금씩, 크면 많이 하게 됨. 크게하면 학습데이터에 너무 맞아 과대적합 될 수 있음

## GradientBoosting
- 대용량 데이터일 때, 성능이 좋음
- 개별 모델로 Decision Tree 사용
- depth가 깊지 않은 트리를 많이 연결해, 이전 트리의 오차를 보정해 나가는 방식
    - 성능이 좋은 모델을 사용하면, 오히려 과대적합 날 수 있음
- 오차를 보정할 때 경사하강법(Gradient descent) 사용(미분)
- 얕은 트리를 많이 연결해, 각 트리가 데이터 일부에 대해 예측 잘 수행하도록 하고 그런 트리들이 모여 전체 성능 높이는 형태
- 분류와 회귀 둘다 지원하는 모델 (GradientBoostingClassification, GrandientBoostingRegressor)
- 훈련시간 많이 걸리고, 트리기반 모델의 특성상 희소한 고차원 데이터에서는 성능 좋지 않은 단점
    - 희소한 고차원 데이터? 값에 0이 많은, 컬럼이 많은 데이터 => 고차원의 희소행렬

### 주요 파라미터
- Decision Tree의 가지치기 관련 매개변수
    - 각각 tree가 복잡한 모델이 되지 않도록 함
- learning rate
    - 이전 tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값
    - 값이 크면 보정 강하게 해 복잡한 모델 만듬. 학습데이터 정확도는 올라가지만 과대적합 날 수 있음 
    - 값을 작게 잡으면 보정 약하게 하여 모델 복잡도 줄임. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수 있음
    - 기본값 : 0.1, 0.01에서 보통 시작해 늘리거나, 줄여나가는 형식
- n_estimators
    - tree 개수 지정. 많을수록 복잡한 모델됨
    - Learning rate와 관련(반비례)
        - Learning rate를 크게 주면 -> n_estimators는 작게
        - Learning rate를 작게 주면 -> n_estimators는 크게
- n_iter_no_change(정수), validation_fraction(비율)
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수동안 검증점수가 좋아지지 않으면 훈련을 조기 종료(early stopping
        - validation_fraction을 0.1, n_iter_no_change = 5, n_estimators = 1000인 상황. 오차 찾는 과정에서 tree 300개로 이미 오차 다 찾음. 나멎 700개 트리는 오차가 없어서 할 일이 없음. 300개에서 끊어버리면 됨  
        - validation_fraction: 학습률의 증가치. 5개 모델 학습하는데 성능 높아지는 비율(오차 줄어드는 비율)이 10% 이상 증가하지 않으면 트리가 더 남아있다고 하더라도 더 이상 학습하지 말고 정지

- 보통 max_depth 낮춰 개별 트리 복잡도 낮춤(5가 넘지 않게). 그리고 n_estimators를 가용시간, 메모리 한도에 맞춘 뒤, 적절한 learning_rate 찾음

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [4]:
data = load_breast_cancer()
X, y = data['data'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 1)
# GradientBoosting 하기에는 데이터 양 적은 편 -> 복잡한 모델이기 때문에 과적합 날 가능성 높음
y_train.shape, y_test.shape

((426,), (143,))

In [ ]:
GradientBoosting?

In [6]:
gb = GradientBoostingClassifier(random_state = 1)
gb.fit(X_train, y_train)

GradientBoostingClassifier(random_state=1)

In [7]:
pred_train = gb.predict(X_train)
pred_test = gb.predict(X_test)

In [ ]:
accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

In [8]:
# feature 중요도
import pandas as pd
fi = gb.feature_importances_
fi_s = pd.Series(fi, index = data['feature_names'])

In [9]:
fi_s.sort_values(ascending = False)

worst radius               0.383760
worst concave points       0.285992
worst perimeter            0.130660
mean concave points        0.046544
worst area                 0.042472
worst texture              0.041142
worst concavity            0.013033
area error                 0.010910
mean texture               0.009101
mean concavity             0.007719
radius error               0.004902
concavity error            0.003397
worst fractal dimension    0.002838
mean area                  0.002552
worst symmetry             0.002546
fractal dimension error    0.002532
mean compactness           0.001566
compactness error          0.001386
mean perimeter             0.001368
smoothness error           0.001340
symmetry error             0.001265
perimeter error            0.000813
mean radius                0.000701
mean fractal dimension     0.000625
texture error              0.000498
worst smoothness           0.000144
worst compactness          0.000097
mean smoothness            0

### GridSearchCV 이용해 최적의 하이퍼파라미터 찾기

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param = {
    'n_estimators': [100, 200, 300, 400, 500],  # tree개수(default: 100)
    'learning_rate': [0.001, 0.005, 0.01, 0.5, 0.1],  # 학습률
    'max_depth': range(1, 5),
    'subsample': [0.5, 0.7, 1],  # 학습시킬 sample의 비율
}
gb = GradientBoostingClassifier(random_state = 1)

gs = GridSearchCV(gb,
                  param_grid = param,
                  cv = 3,
                  scoring = 'accuracy',
                  n_jobs = -1)

In [11]:
gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=1),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.005, 0.01, 0.5, 0.1],
                         'max_depth': range(1, 5),
                         'n_estimators': [100, 200, 300, 400, 500],
                         'subsample': [0.5, 0.7, 1]},
             scoring='accuracy')

In [12]:
gs.best_params_

{'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 100, 'subsample': 0.5}

In [15]:
result_df = pd.DataFrame(gs.cv_results_)
result_df.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
215,1.249222,0.013107,0.002982,0.000816,0.5,3,200,1,"{'learning_rate': 0.5, 'max_depth': 3, 'n_esti...",0.964789,0.971831,0.985915,0.974178,0.008783,1
195,0.377336,0.024049,0.001508,0.000722,0.5,2,100,0.5,"{'learning_rate': 0.5, 'max_depth': 2, 'n_esti...",0.964789,0.978873,0.978873,0.974178,0.006640,1
207,1.562981,0.013348,0.002830,0.000229,0.5,2,500,0.5,"{'learning_rate': 0.5, 'max_depth': 2, 'n_esti...",0.964789,0.978873,0.978873,0.974178,0.006640,1
263,1.402380,0.052528,0.002329,0.000471,0.1,2,300,1,"{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",0.957746,0.971831,0.992958,0.974178,0.014470,1
289,1.292763,0.050067,0.002328,0.000470,0.1,4,200,0.7,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.964789,0.971831,0.985915,0.974178,0.008783,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,0.812115,0.051518,0.001836,0.000224,0.001,2,200,0.7,"{'learning_rate': 0.001, 'max_depth': 2, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277
45,0.626055,0.120975,0.002327,0.000470,0.001,4,100,0.5,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277
17,0.887718,0.015895,0.001663,0.000470,0.001,2,100,1,"{'learning_rate': 0.001, 'max_depth': 2, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277
48,1.008480,0.024664,0.002327,0.000471,0.001,4,200,0.5,"{'learning_rate': 0.001, 'max_depth': 4, 'n_es...",0.626761,0.626761,0.626761,0.626761,0.000000,277


In [16]:
pred_test = gs.predict(X_test)
accuracy_score(y_test, pred_test)

0.9440559440559441

In [17]:
model = gs.best_estimator_
model.feature_importances_

array([0.00000000e+00, 1.33549096e-03, 4.94494698e-03, 0.00000000e+00,
       3.97585690e-06, 9.34141518e-02, 6.51982210e-03, 7.02625571e-02,
       1.20680063e-01, 8.13146442e-03, 1.05668682e-03, 1.65773029e-02,
       0.00000000e+00, 4.81324605e-03, 9.02874999e-03, 3.78019977e-05,
       2.79283962e-04, 0.00000000e+00, 1.49012865e-01, 3.10905613e-05,
       2.23908297e-01, 2.81342931e-02, 4.66786793e-02, 1.83046772e-03,
       6.69333072e-03, 7.19996363e-06, 4.34145107e-03, 4.09313717e-02,
       8.70474544e-02, 7.42979558e-02])

# XGBoost(Extra Gradient Boost)
- https://xgboost.readthedocs.io/
- Gradient Boost 알고리즘을 기반으로 개선해서 나온 모델.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것으로 알려저 유명해짐
- Gradient Boost의 단점인 느린수행시간을 해결하고 과적합을 제어할 수 있는 규제를 제공해 성능 높임
- 두가지 개발 방법
    - [Scikit-learn 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)
    - [파이썬 래퍼 XGBoost 모듈 사용](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training)
- 설치   
``
pip install xgboost
conda install -y -c anaconda py-xgboost
``



In [1]:
# !pip install xgboost

## Scikit-learn 래퍼 XGBoost
- XGBoost를 Scikit-learn프레임워크와 연동할 수 있도록 개발됨.
- Scikit-learn의 Estimator들과 동일한 패턴으로 코드를 작성할 수 있다.
- GridSearchCV나 Pipeline 등 Scikit-learn이 제공하는 다양한 유틸리티들을 사용할 수 있다.
- XGBClassifier: 분류
- XGBRegressor : 회귀 

### 주요 매개변수
- learning_rate : 학습률, 보통 0.01 ~ 0.2 사이의 값 사용
- n_estimators : week tree 개수
- max_depth: 트리의 depth 지정.

In [18]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators = 200,
                    learning_rate = 0.5,
                    max_depth = 2)

In [19]:
xgb.fit(X_train, y_train)

XGBClassifier(learning_rate=0.5, max_depth=2, n_estimators=200)

In [20]:
pred_train = xgb.predict(X_train)
pred_test = xgb.predict(X_test)

In [21]:
accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

(1.0, 0.965034965034965)

In [22]:
# feature importance
fi = xgb.feature_importances_
fi

array([0.0000000e+00, 1.7377844e-02, 0.0000000e+00, 2.7057502e-02,
       9.4335829e-04, 5.7103140e-03, 3.3941608e-02, 9.9188931e-02,
       4.1985163e-03, 2.1932845e-03, 0.0000000e+00, 3.3285678e-04,
       1.7986957e-02, 1.1520459e-02, 0.0000000e+00, 2.2246058e-03,
       0.0000000e+00, 1.2838596e-03, 9.7346597e-04, 8.4046111e-04,
       2.2470552e-01, 1.2404799e-02, 1.2866795e-01, 4.5809176e-02,
       4.1861958e-03, 1.6558268e-04, 1.2857459e-02, 3.3936533e-01,
       4.8814113e-03, 1.1825919e-03], dtype=float32)